In [249]:
import pandas as pd
import os
import re  
import numpy as np
import matplotlib.pyplot as plt  
# 函数定义

def get_filenames_in_dir(directory):  
    return os.listdir(directory)  
    
def find_closest_number(df, target_number):  
    closest_column = None  
    closest_diff = float('inf')  
      
    for column in df.columns:  
        for value in df[column]:  
            diff = abs(value - target_number)  
            if diff < closest_diff:  
                closest_diff = diff  
                closest_column = column  
                break  # 找到最接近的值后，退出循环以节省时间  
      
    if closest_column is not None:  
        return closest_column, df[closest_column][0]  
    else:  
        return None, None
def extract_integer(string):  
    # 使用正则表达式匹配数字部分  
    match = re.search(r'\d+', string)  
    if match:  
        # 如果找到数字部分，将其转换为整数型元素  
        return int(match.group())  
    else:  
        return None  # 如果没有找到数字部分，返回None  
def get_csv_filename_prefix(file_path):  
    # 从文件路径中提取文件名  
    filename = file_path.split('/')[-1]  
    # 提取文件名中'.csv'之前的部分  
    prefix = filename.rsplit('.', 1)[0]  
    return prefix  
def get_expower(file_name):
    # 寻找ms_的位置
    ms_index = file_name.find('ms_')
    
    # 如果找到ms_，则获取ms_后面到下一个_之间的字符作为功率值
    if ms_index != -1:
        start_index = ms_index + len('ms_')
        next_underscore_index = file_name.find('_', start_index)
        
        if next_underscore_index != -1:
            power = file_name[start_index:next_underscore_index]
            return power
        else:
            # 如果没有找到下一个_，则返回错误或者默认值，这里返回None
            return None
    else:
        # 如果没有找到ms_，则返回错误或者默认值，这里返回None
        return None

def get_co2_power(file_name):
    # 寻找W_的位置
    w_index = file_name.find('W_')
    
    # 如果找到W_，则获取W_后面三个字符作为CO2功率值
    if w_index != -1:
        start_index = w_index + len('W_')
        co2_power = file_name[start_index:start_index + 3]
        return co2_power
    else:
        # 如果没有找到W_，则返回错误或者默认值，这里返回None
        return None

# 测试
# file_name = "NdYAG_730Exc_100ms_50uW_4_5"
# co2_power = get_co2_power(file_name)
# print(co2_power)


# 设置工作路径

In [250]:
# 设置工作路径
directory = 'D:/HanyuZhang/730exNd1114/8mW'
os.chdir(directory)
# print(os.getcwd())

# 获取文件夹内的文件名列表  
file_names = get_filenames_in_dir(directory)  


def load_file_as_df(file_name):  
    # 使用os.path.normpath()来规范化路径  
    normalized_path = os.path.normpath(file_name)  
    # 使用pandas的read_csv函数加载数据  
    data = pd.read_csv(normalized_path, delimiter='\t', header=None)  
    return pd.DataFrame(data)
df_list = []  
for file_name in file_names:  
    df_list.append(load_file_as_df(file_name))

# df_list[0]

In [251]:
# df_list

In [252]:
def process_data(df):  
    # 获取波长列表  
    wav = df[df.index==1]  
    wav_split = wav.iloc[:,0].str.split(',')  
    new_df = pd.DataFrame(wav_split.tolist(), columns=[f'Column{i}' for i in range(len(wav_split.iloc[0]))])  
    wav_df = new_df.apply(pd.to_numeric,errors='coerce')  
  
    # 获取强度列表（所有Frame合并为一个list）  
    n_values = df.index // 3  
    selected_rows = df[df.index == 2 + 3 * n_values]  
    df = pd.DataFrame(selected_rows)  
    df_split = df.iloc[:,0].str.split(',')  
    new_df = pd.DataFrame(df_split.tolist(), columns=[f'Column{i}' for i in range(len(df_split.iloc[0]))])  
    newnew_df = new_df.apply(pd.to_numeric,errors='coerce')  
  
    # 选择需要求和的波长范围  
    left = extract_integer(find_closest_number(wav_df, 790)[0])  
    right = extract_integer(find_closest_number(wav_df, 825)[0])  
    newnew_df['sum'] = newnew_df.iloc[:, left:right].sum(axis=1)  
  
    max_value = newnew_df['sum'].max()  
    normal = newnew_df['sum'] / max_value  
  
    return normal

In [253]:
# df_list[0]

In [254]:
import pandas as pd
import matplotlib.pyplot as plt

# 创建一个空的 DataFrame 用于存储结果
result_df = pd.DataFrame(columns=["power", "increase"])
for i, data in enumerate(df_list):  
    # 处理数据并获取图形数据  
    normal = process_data(data)  
    # 绘制图形  
    name = get_csv_filename_prefix(file_names[i])
    power = get_co2_power(name)
    # power = get_power(name)
    # print(power)
    # plt.plot(normal, label=power)  
    result = []  
    for j in range(len(normal)):  
        if j < len(normal) - 10:  # 如果当前元素不是最后一个元素  
            result.append(normal[j+10] - normal[j])  # 计算往后10个元素减去当前元素的值，并添加到结果列表中  
        else:  # 如果当前元素是最后一个元素  
            result.append(normal[len(normal)-1] - normal[j])  # 使用最后一个元素减去当前元素的值作为替代 
    co2on = result.index(max(result)) # 找到max(result)的位置  
    # co2off = result.index(min(result)) # 找到min(result)的位置  
    delta = normal[co2on + 20] - normal[co2on]
    delta_div_co2on = delta / normal[co2on]

    # 将结果添加到 DataFrame
    result_df = pd.concat([result_df, pd.DataFrame({"power": [power], "increase": [delta_div_co2on]})], ignore_index=True)
    print(power, delta/normal[co2on])
    print(co2on)

# 将结果保存到 CSV 文件
result_df.to_csv(get_expower(file_names[0]) + '.csv', index=False)



C:\Users\Administrator\AppData\Local\Temp\ipykernel_9624\200194297.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({"power": [power], "increase": [delta_div_co2on]})], ignore_index=True)


2_5 0.1934833130746885
47
3_0 0.511960368422582
42
3_5 0.652072754515941
52
4_0 0.9089926693853836
53
4_5 1.2080091147722007
61


In [255]:
import os
import pandas as pd

# 设置文件夹路径
folder_path = 'D:/HanyuZhang/730exNd1114/merge'  # 替换为实际的文件夹路径

# 获取文件夹中的所有文件名
file_names = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# 初始化一个 DataFrame 用于存储结果
result_df = pd.DataFrame()

# 遍历文件夹中的每个文件
for file_name in file_names:
    # 生成文件的完整路径
    file_path = os.path.join(folder_path, file_name)

    # 读取文件的第二列
    data = pd.read_csv(file_path, usecols=[1], header=None)

    # 将第二列数据添加到结果 DataFrame，以文件名（不带扩展名）作为列名
    result_df[file_name.split('.')[0]] = data.iloc[:, 0]

# 将结果保存到新的 CSV 文件
result_df.to_csv("merged_result.csv", index=False)
